# 06_saving_consecutive_metrics_all_models
* Calculating the consecutive metrics for all models and then saving this to a folder with in the consecutve metrics folder with the folder name being the window length.

PHD-10
* Newer more streamlined version from Documents/PhD/03_SN_global_different_windows_(nb25).ipynb
* ALl S/N calculation and bound are handled by function

# Preamble

In [3]:
import numpy as np
import os
from time import perf_counter
import itertools
import matplotlib.pyplot as plt
import xarray as xr
from importlib import reload
from glob import glob
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from enum import Enum
from dask.diagnostics import ProgressBar
from typing import Dict, List

In [4]:
import json

with open('data/longrunmip_model_lengths.json') as f:
    longrunmip_model_lengths = json.loads(f.read())
    
longrunmip_model_lengths['good_models']

{'ccsm3': {'control': 1530, '4xCO2': 2120},
 'cesm104': {'control': 1000, '4xCO2': 5900},
 'cnrmcm61': {'control': 2000, '4xCO2': 1850},
 'famous': {'control': 3000, '4xCO2': 3000},
 'gisse2r': {'control': 5225, '4xCO2': 5001},
 'hadcm3l': {'control': 1000, '4xCO2': 1000},
 'ipslcm5a': {'control': 1000, '4xCO2': 1000},
 'mpiesm11': {'control': 2000, '4xCO2': 4459},
 'mpiesm12': {'control': 1237, '4xCO2': 1000}}

In [6]:
import sys
sys.path.append('modules')
# Custom xarray classes that addes different method.
import xarray_class_accessors as xca

# Custom plots for signal to noise functions.
import sn_plotting
import signal_to_noise as sn
import open_ds
import constants

In [7]:
constants.LONGRUNMIP_CONSECMET_DIR

'/g/data/w40/ab2313/PhD/longrunmip/consecutive_metrics'

In [8]:
# The directory where all of the longrun mip data used in this notebook is stored.
LONGRUNMIP_RETIMED_DIR = constants.LONGRUNMIP_RETIMED_DIR
LONGRUNMIP_RETIMED_DIR

'/g/data/w40/ab2313/PhD/longrunmip/regrid_retimestamped'

In [9]:
# It takes a while to run the loess filter for each grid cell. So this has been pre-generated and saved.
LONRUNMIP_LOESS_DIR = constants.LONRUNMIP_LOESS_DIR
LONRUNMIP_LOESS_DIR

'/g/data/w40/ab2313/PhD/longrunmip/regrid_retimestamped/loess'

In [10]:
constants.LONGRUNMIP_DIR

'/g/data/w40/ab2313/PhD/longrunmip'

In [11]:
chunks = {'lat':48/2,'lon':96/2,'time':100}

# File Names

In [12]:
list(longrunmip_model_lengths['good_models'])

['ccsm3',
 'cesm104',
 'cnrmcm61',
 'famous',
 'gisse2r',
 'hadcm3l',
 'ipslcm5a',
 'mpiesm11',
 'mpiesm12']

In [13]:
list(longrunmip_model_lengths['bad_models'])

['ecearth', 'miroc32']

In [14]:
FILE_NAME_DICT = open_ds.get_exeriment_file_names()

abrupt4x_raw
ECHAM5MPIOM, ECEARTH, HadGEM2, MIROC32, GFDLCM3, GFDLESM2M, 
- Fraction of good models 1.0
------
control_raw
HadGEM2, GFDLCM3, ECHAM5MPIOM, GFDLESM2M, GFDLESM2M, ECEARTH, GFDLCM3, MIROC32, 
- Fraction of good models 1.0
------
abrupt4x_loess
ECHAM5MPIOM, GFDLESM2M, MIROC32, GFDLCM3, HadGEM2, ECEARTH, 
- Fraction of good models 1.0
------
control_loess
GFDLESM2M, ECEARTH, GFDLCM3, MIROC32, HadGEM2, ECHAM5MPIOM, 
- Fraction of good models 1.0
------


In [15]:
FILE_NAME_DICT.keys()

dict_keys(['abrupt4x_raw', 'control_raw', 'abrupt4x_loess', 'control_loess'])

In [16]:
FILE_NAME_DICT['abrupt4x_raw']['file_names']

['tas_mon_GISSE2R_abrupt4x_5001_g025.nc',
 'tas_mon_MPIESM12_abrupt4x_999_g025.nc',
 'tas_mon_CNRMCM61_abrupt4x_1850_g025.nc',
 'tas_mon_CCSM3_abrupt4x_2120_g025.nc',
 'tas_ann_MPIESM11_abrupt4x_4459_g025.nc',
 'tas_mon_CESM104_abrupt4x_5900_g025.nc',
 'tas_ann_FAMOUS_abrupt4x_3000_g025.nc',
 'tas_mon_HadCM3L_abrupt4x_1000_g025.nc',
 'tas_mon_IPSLCM5A_abrupt4x_1000_g025.nc']

In [17]:
assert len(FILE_NAME_DICT['abrupt4x_raw']['file_names']) == len(list(longrunmip_model_lengths['good_models']))

In [18]:
# List of all the models that have an experiment
models  = [f.split('_')[2] for f in FILE_NAME_DICT['abrupt4x_raw']['file_names']]
models

['GISSE2R',
 'MPIESM12',
 'CNRMCM61',
 'CCSM3',
 'MPIESM11',
 'CESM104',
 'FAMOUS',
 'HadCM3L',
 'IPSLCM5A']

# Procssing and Saving All Models

In [19]:
FILE_NAME_DICT.keys()

dict_keys(['abrupt4x_raw', 'control_raw', 'abrupt4x_loess', 'control_loess'])

In [20]:
model_fnames = open_ds.get_all_file_names_for_model(models[2], FILE_NAME_DICT)
model_fnames

{'abrupt4x_raw': 'tas_mon_CNRMCM61_abrupt4x_1850_g025.nc',
 'control_raw': 'tas_mon_CNRMCM61_control_2000_g025.nc',
 'abrupt4x_loess': 'tas_mon_CNRMCM61_abrupt4x_1850_g025_loess.nc',
 'control_loess': 'tas_mon_CNRMCM61_control_2000_g025_loess.nc'}

In [32]:
RUN_PARAMETERS = {'window': 101, 'requested_length': 500}

In [33]:
output_path = os.path.join(
        constants.LONGRUNMIP_CONSECMET_DIR, 
        f"window_{RUN_PARAMETERS['window']}_length_{RUN_PARAMETERS['requested_length']}"
    )
output_path

'/g/data/w40/ab2313/PhD/longrunmip/consecutive_metrics/window_101_length_500'

In [34]:
os.path.isdir(output_path)

True

In [43]:
completed_files = list(map(os.path.basename, glob(output_path + '/*da_sn_[!stable]*')))
completed_files

['da_sn_CNRMCM61.nc',
 'da_sn_MPIESM12.nc',
 'da_sn_GISSE2R.nc',
 'da_sn_CCSM3.nc']

In [44]:
completed_models = [v.split('.')[0].split('_')[2]for v in completed_files]
completed_models

['CNRMCM61', 'MPIESM12', 'GISSE2R', 'CCSM3']

In [45]:
models

['GISSE2R',
 'MPIESM12',
 'CNRMCM61',
 'CCSM3',
 'MPIESM11',
 'CESM104',
 'FAMOUS',
 'HadCM3L',
 'IPSLCM5A']

In [47]:
needed_models = [v for  v in models if v not in completed_models]
needed_models

['MPIESM11', 'CESM104', 'FAMOUS', 'HadCM3L', 'IPSLCM5A']

In [48]:
RUN_PARAMETERS

{'window': 101, 'requested_length': 500}

In [49]:
def open_dataarray_subset(DIR, fname, RUN_PARAMETERS):
    fpath = os.path.join(DIR, fname)
    print(f'- Opening {fpath}')
    da = xr.open_dataset( fpath,use_cftime = True, chunks = chunks).tas
    # Need to go +window past length or final points will be cut off.
    true_length = RUN_PARAMETERS['requested_length'] + RUN_PARAMETERS['window']
    da = da.isel(time=slice(0, true_length))
    
    print(f'    + Needed length  - {true_length} - Opened length - {len(da.time.values)}')
    return da.compute()                 

In [50]:
for i,model in enumerate(needed_models):
    t_start = perf_counter()
    print(f'Getting data for {model} with parameters {RUN_PARAMETERS} ({i+1}/{len(needed_models)})')
    model_fnames = open_ds.get_all_file_names_for_model(model, FILE_NAME_DICT)
    

    # Opening data for control and experiment
    single_experiment = open_dataarray_subset(
        LONGRUNMIP_RETIMED_DIR,model_fnames['abrupt4x_raw'], 
        RUN_PARAMETERS)
    
    single_control = open_dataarray_subset(
        LONGRUNMIP_RETIMED_DIR,model_fnames['control_raw'], 
        RUN_PARAMETERS)
    
    single_experiment_loess = open_dataarray_subset(
        LONRUNMIP_LOESS_DIR,model_fnames['abrupt4x_loess'], 
        RUN_PARAMETERS)
    
    single_control_loess = open_dataarray_subset(
        LONRUNMIP_LOESS_DIR,model_fnames['control_loess'], 
        RUN_PARAMETERS)
    
    
    # Getting the anomalies for the experiment
    print('--------')
    experiment_anom = single_experiment.clima.anomalies(start = single_experiment.time.dt.year.values[0], 
                                              end = single_experiment.time.dt.year.values[-1],
                                historical = single_experiment)

    print('Calculating signal to noise')
    # Signal to noise
    ds_sn, ds_sn_stable, ds_sn_increasing, ds_sn_decreasing =\
            sn.global_sn(experiment_anom, single_control, 
                         single_experiment_loess, single_control_loess, 
                         window=RUN_PARAMETERS['window'])
    

    print('Computing signal to noise')
    ds_sn = ds_sn.compute()
    ds_sn_stable = ds_sn_stable.compute()

    
    # Consecutive metrics
    print('Calculating consecutive metrics')
    stable_consec_mets_ds = ds_sn_stable['signal_to_noise'].sn.calculate_consecutive_metrics()
    print('Computing consecutve metrics')
    stable_consec_mets_ds = stable_consec_mets_ds.compute()
    
    # Saving
    print('--------')
    print(f'Saving to {output_path}')
    
    consec_save_name = os.path.join(output_path, f'consec_mets_{model}.nc')
    print(f'    + {consec_save_name}')
    stable_consec_mets_ds.to_netcdf(consec_save_name)
    
    stable_save_name = os.path.join(output_path, f'da_sn_stable_{model}.nc')
    print(f'    + {stable_save_name}')
    ds_sn_stable.to_netcdf(stable_save_name)
    
    sn_save_name = os.path.join(output_path, f'da_sn_{model}.nc')
    print(f'    + {sn_save_name}')
    ds_sn.to_netcdf(sn_save_name)

    t_final = perf_counter()
    print(f'- Time taken: {t_final - t_start} seconds')
    print('\n\n')


Getting data for MPIESM11 with parameters {'window': 101, 'requested_length': 500} (1/5)
- Opening /g/data/w40/ab2313/PhD/longrunmip/regrid_retimestamped/tas_ann_MPIESM11_abrupt4x_4459_g025.nc
    + Needed length  - 601 - Opened length - 601
- Opening /g/data/w40/ab2313/PhD/longrunmip/regrid_retimestamped/tas_ann_MPIESM11_control_2000_g025.nc
    + Needed length  - 601 - Opened length - 601
- Opening /g/data/w40/ab2313/PhD/longrunmip/regrid_retimestamped/loess/tas_ann_MPIESM11_abrupt4x_4459_g025_loess.nc
    + Needed length  - 601 - Opened length - 601
- Opening /g/data/w40/ab2313/PhD/longrunmip/regrid_retimestamped/loess/tas_ann_MPIESM11_control_2000_g025_loess.nc
    + Needed length  - 601 - Opened length - 601
--------
Using historical dataset
Calculating signal to noise
Computing signal to noise
Calculating consecutive metrics
New data has shape (5, 72, 144)
Computing consecutve metrics
--------
Saving to /g/data/w40/ab2313/PhD/longrunmip/consecutive_metrics/window_101_length_500
 